# First attempt at learning

In [ ]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()
import os
import random
from tqdm import tqdm

SELECTED_DATA_DIR = "../selected-data/"
MOVIES_FILE = "best_movie_ratings_features_engineered.csv"
USERS_FILE = "users_ratings.csv"

## Read data

In [ ]:
movies = pd.read_csv(SELECTED_DATA_DIR + MOVIES_FILE, index_col=0)
movies.rating = movies.rating/10
movies.sample()

In [ ]:
users = pd.read_csv(SELECTED_DATA_DIR + USERS_FILE, index_col=0)
users.rating = users.rating/10
users.sample()

## Random selection of a user

In [ ]:
user = users[users.user.isin(users.user.sample())]
user.shape

In [ ]:
# get only movies that this user rated
movies_user = movies[movies.index.isin(user.index)]
movies_user.shape

## Learning

In [ ]:
s = 1
def utility(user_features, movie_features):
    """ Compute utility based on user preferences and movie preferences """
    return user_features.dot(movie_features)

def get_movie_features(movie):
    """ selected features from dataframe """
    if isinstance(movie, pd.Series):
        return movie[2:]
    elif isinstance(movie, pd.DataFrame):
        return get_movie_features(movie.loc[movie.index[0]])
    else:
        raise TypeError("{} should be a Series or DataFrame".format(movie))
    
def best_recommandation(user_features, movies):
    """ Return the movie with the highest utility """
    utilities = np.zeros(movies.shape[0])
    for i, (title, movie) in enumerate(movies.iterrows()):
        movie_features = get_movie_features(movie)
        utilities[i] = utility(user_features, movie_features)
    return movies[movies.index == movies.index[utilities.argmax()]]
    
def greedy_choice(user_features, movies, epsilon=1):
    """ greedy approach to the problem """
    if random.random() > epsilon: # choose the best
        return best_recommandation(user_features, movies)
    else:
        return movies.sample()
        
def iterative_mean(old, new, t):
    """ Compute the new mean """
    return ((t-1) / t) * old + (1/t) * new
    
def update_features(user_features, movie_features, rating, t):
    """ update the user preferen """
    return iterative_mean(user_features, movie_features * rating, t)

def train_user(user, movies):
    user_features = np.zeros(movies.shape[1] - 2)
    for t in tqdm(range(1, 10000)):
        recommandation = greedy_choice(user_features, movies)
        recommandation_features = get_movie_features(recommandation)
        user_rating = user.get_value(recommandation.index[0], "rating")
        user_features = update_features(user_features, recommandation_features, user_rating, t)
    return user_features

user_features = train_user(user, movies_user)
user_features

In [ ]:
recomm = best_recommandation(user_features, movies)
recomm

In [ ]:
recomm.as_matrix()

In [ ]:
user_features